# Cell type identification from scRNA-seq data using ACTINN

## Why is RNA important?

To understand what scRNA-seq data represents, let's first explore the role of RNA (Ribonucleic Acid) inside cells. All cells in our body, regardless of whether they make up tissues in the heart, brain, or skin, contain the same genetic code (DNA). Yet, despite having identical DNA, cells look different, perform distinct functions, and form specialized tissues and organs.

How is this possible?

The key lies in differential [gene expression](https://en.wikipedia.org/wiki/Gene_expression), the process by which genetic instructions are used to synthesize gene products like proteins. DNA acts as the blueprint, but it is RNA, specifically messenger RNA (mRNA), that translates these instructions into action. Different types of cells transcribe distinct subsets of genes into mRNAs, leading to the synthesis of unique sets of proteins. These distinct protein profiles allow cells to specialize and perform various functions within the body [1,2].

Messenger RNA (mRNA) acts as a critical intermediary, carrying genetic instructions from DNA to ribosomes,the cellular machinery responsible for protein synthesis.

Proteins like enzymes drive metabolic pathways, while structural proteins such as actin and myosin give muscles their ability to contract and move. Cytoskeletal proteins help maintain cell shape and integrity.

Therefore, the unique set of mRNA molecules present in a cell at any given moment(its **transcriptome**) reveals which genes are "turned on." This transcriptome dictates which proteins the cell is making, which in turn defines the cell's type and function. Measuring this transcriptome is the fundamental goal of single-cell RNA sequencing (scRNA-seq).


## Single-cell RNA sequencing vs Bulk RNA sequencing

RNA-Seq (short for RNA sequencing) is a technique used to quantify and identify RNA molecules in a biological sample, providing a snapshot of the transcriptome at a specific time. Sequencing of RNA can be mainly conducted in two ways: Either by sequencing the mixed RNA from the source of interest across cells (bulk sequencing) or by sequencing the transcriptomes of the cells individually (single-cell sequencing).

Bulk RNA-Seq results in cell-averaged expression profiles, which are generally easier to analyze, but also hide some of the complexity such as cell expression profile heterogeneity, which may help answer the question of interest. Some drugs or perturbations may affect only specific cell types or interactions between cell types.

For example, in oncology, it is possible to have rare drug resistant tumor cells causing relapse, which is difficult to identify by simple bulk RNA-seq even on cultured cells.To uncover such relationships, it is vital to examine gene expression on a single-cell level.[3]

## Libraries commonly used for scRNA-seq analysis

[ScanPy](https://scanpy.readthedocs.io/en/stable/) is a scalable toolkit for analyzing single-cell gene expression data. It includes methods for preprocessing, visualization, clustering, pseudotime and trajectory inference, differential expression testing, and simulation of gene regulatory networks.

[ANNDATA](https://github.com/theislab/anndata) was presented alongside ScanPy as a generic class for handling annotated data matrices that can deal with the sparsity inherent in gene expression data.

This is also a good point to explore DeepChem's [tutorial](https://deepchem.io/tutorials/scanpy/) on building an scRNA-seq analysis pipeline using ScanPy.




## ACTINN (automated identification of cell types in single cell RNA sequencing )

This tutorial guides you through working with scRNA-seq data and demonstrates how to train and evaluate a model using DeepChem. The objective is to replicate the experiments performed for cell type identification in [ACTINN](https://academic.oup.com/bioinformatics/article/36/2/533/5540320).

Cell type identification is one of the major goals in single cell RNA sequencing (scRNA-seq). Traditional methods for assigning cell types typically involve the use of unsupervised clustering, the identification of signature genes in each cluster, followed by a manual lookup of these genes in the literature and databases to assign cell types. However, there are several limitations associated with these approaches, such as unwanted sources of variation that influence clustering and a lack of canonical markers for certain cell types.[1]

ACTINN employs a neural network with three hidden layers, trains on datasets with predefined cell types and predicts cell types for other datasets based on the trained parameters.

Here, ACTINN is implemented with a deepchem wrapper.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Advanced_Model_Training.ipynb)

## Setup

To run DeepChem within Colab, you'll need to run the following installation commands. You can of course run this tutorial locally if you prefer. In that case, don't run these cells since they will download and install DeepChem in your local machine again.

In [2]:
!pip install deepchem
import deepchem as dc
dc.__version__

  Using cached deepchem-2.8.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 40.2 MB/s eta 0:00:00


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.0'

## Data Collection and Splitting




A subset of the 'tma_ss2_cleaned' dataset from this [source](https://figshare.com/articles/ACTINN/8967116), which was originally used in ACTINN, is utilized here.

There are two approaches to creating the train and test sets:
1. Splitting a single dataset into training and testing portions.
2. Using datasets from different sources as the train and test sets.


In [3]:
import os
current_dir = os.path.dirname(os.path.realpath('__file__'))

In [4]:
file_names = ['scRNAseq_sample_1.h5', 'scRNAseq_sample_2.h5', 'celltype_label_sample_1.txt']
base_url = 'https://raw.githubusercontent.com/Harindhar10/deepchem/actinn/deepchem/feat/tests/data/sc_rna_seq_data'

for i in range(len(file_names)):
  dc.utils.download_url(
      f"{base_url}/{file_names[i]}",
      current_dir,
      f"{file_names[i]}"
  )

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### 1. Same source

In [6]:
labels = pd.read_csv(os.path.join(current_dir,'celltype_label_sample_1.txt'), header=None, sep='\t')
dataset = pd.read_hdf(os.path.join(current_dir,'scRNAseq_sample_1.h5'))

In [7]:
dataset

,tma_mfd_1808,tma_mfd_32608,tma_mfd_2589,tma_mfd_13999,tma_mfd_2621,tma_mfd_5055,tma_mfd_796,tma_mfd_26185,tma_mfd_30053,tma_mfd_8084,...,tma_mfd_187,tma_mfd_5673,tma_mfd_6170,tma_mfd_34321,tma_mfd_8061,tma_mfd_2459,tma_mfd_13712,tma_mfd_33269,tma_mfd_32996,tma_mfd_33381
0610005C13Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0610007C21Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
0610007L01Rik,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0610007N19Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0610007P08Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Anxa5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Anxa6,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0
Anxa7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
Anxa8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


'labels' has columns cell ID and cell type

In [8]:
n_types = 0 # To store the number of cell types present in the dataset, which is need to define the output layer of the model later on.

# Convertes cell type labels from string ('B Cell', 'T Cell' etc) to integers (1, 2 etc)
def convert_type2label(types):
    global n_types
    types = list(types)
    unique_types = sorted(set(types)) # Sorting ensures consistent label order
    n_types = len(unique_types)

    type_to_label_dict = {t: i for i, t in enumerate(unique_types)}
    labels = np.array([type_to_label_dict[t] for t in types])

    return labels

labels_int = convert_type2label(labels[1])

In [9]:
# Reshaping labels from (n,) to (n, 1) to be compatible with DeepChem's Dataset class
labels_int = labels_int.reshape(labels_int.shape[0],1)

In [10]:
# gene IDs are stored
train_genes = dataset.index.to_numpy()

# genes x cells (features x samples) -> cells x genes (samples x features)
dataset = np.transpose(dataset)

# Creating Deepchem 'Dataset' object
dataset = dc.data.NumpyDataset(X=np.array(dataset,dtype=np.float32),y = labels_int, ids= labels[0])

In [16]:
splitter = dc.splits.SingletaskStratifiedSplitter()
train_dataset, test_dataset = splitter.train_test_split(dataset)

IndexError: arrays used as indices must be of integer (or boolean) type

### 2. Different sources

When using datasets from different sources, the gene sets may not completely overlap. Since the ACTINN model defines its layers based on the gene set, it's essential that both the training and testing sets contain the same genes. To ensure this, we first identified the genes common to both datasets and filtered out those that were not shared.

In [176]:
train_set = pd.read_hdf(os.path.join(current_dir,'scRNAseq_sample_1.h5'))
train_labels = pd.read_csv(os.path.join(current_dir,'celltype_label_sample_1.txt'), header=None, sep='\t')
test_set = pd.read_hdf(os.path.join(current_dir,'scRNAseq_sample_2.h5'))
#test_labels = pd.read_csv(os.path.join(current_dir,'celltype_label_sample_2.txt'), header=None, sep='\t')

common_genes = train_set.index.intersection(test_set.index)
common_genes = sorted(common_genes)

train_set = train_set.loc[common_genes]
test_set = test_set.loc[common_genes]

Convertes cell type labels from string ('B Cell', 'T Cell' etc) to integers (1, 2 etc)

In [148]:
def convert_type2label(types):
    global n_types
    types = list(types)
    unique_types = sorted(set(types)) # Sorting ensures consistent label order
    n_types = len(unique_types)

    type_to_label_dict = {t: i for i, t in enumerate(unique_types)}
    labels = np.array([type_to_label_dict[t] for t in types])

    return labels

train_labels_int = convert_type2label(train_labels[1])
#test_labels_int = convert_type2label(test_labels[1])

In [177]:
# gene IDs are stored
train_genes = train_set.index.to_numpy()
test_genes = test_set.index.to_numpy()

In [178]:
print(len(train_genes),len(test_genes))

802 802


In [179]:
# genes x cells (features x samples) -> cells x genes (samples x features)
train_set = np.transpose(train_set)
test_set = np.transpose(test_set)

In [180]:
# Creating Deepchem 'Dataset' object
train_dataset = dc.data.NumpyDataset(X=np.array(train_set,dtype=np.float32),y = train_labels_int, ids= train_labels[0])
test_dataset = dc.data.NumpyDataset(X=np.array(test_set,dtype=np.float32))

In [181]:
train_dataset

<NumpyDataset X.shape: (1000, 802), y.shape: (1000,), w.shape: (1000,), task_names: [0]>

In [182]:
test_dataset

<NumpyDataset X.shape: (1000, 802), y.shape: (1000, 1), w.shape: (1000, 1), task_names: [0]>

## Train set data transformation and gene filtering

Next, each cell’s expression value was normalized to its total expression value and multiplied by a scale factor of 10 000.

In [183]:
# 1) extract gene names & data array
gene_names = train_genes
X = train_dataset.X

# 2) library-size normalize to 10,000 (in-place)
row_sums = X.sum(axis=1, keepdims=True)  # shape (n_cells, 1)
X /= row_sums # broadcast divide
X *= 10000

In [184]:
X.shape

(1000, 802)

The counts were increased by 1, and the log2 value was calculated

In [185]:
# 3) log2(x + 1) transform (in-place)
X = np.log2(X + 1)

array([[0.       , 0.       , 7.1891084, ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 7.319493 , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 6.792987 , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 4.7355776, ..., 0.       , 0.       ,
        0.       ],
       [0.       , 7.4380746, 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [186]:
X.shape

(1000, 802)

To filter out outlier genes, the genes with the highest 1% and lowest 1% expression were removed.

In [187]:
expr = X.sum(axis=0)
low, high = np.percentile(expr, [1, 99])
mask_expr = (expr >= low) & (expr <= high)


In [188]:
X = X[:, mask_expr]
gene_names = gene_names[mask_expr]

The gene with the highest 1% and the lowest 1% standard deviation were also removed.

In [189]:
mean_expr = X.mean(axis=0)
cv = X.std(axis=0) / mean_expr
low_cv, high_cv = np.percentile(cv, [1, 99])
mask_cv = (cv >= low_cv) & (cv <= high_cv)


In [190]:
X = X[:, mask_cv]
gene_names = gene_names[mask_cv]
train_genes = gene_names

In [191]:
# Creating Deepchem 'Dataset' object
train_dataset = dc.data.NumpyDataset(X=X,y = train_labels_int, ids= train_labels[0])

## Test set transformation and filtering

The exact same normalisation aproach used for train set is used for test set.
First, each cell’s expression value was normalized to its total expression value and multiplied by a scale factor
of 10 000. The counts were increased by 1, and the log2 value was
calculated

In [192]:
# 1) extract data array
X = test_dataset.X

# 2) library-size normalize to 10,000 (in-place)
row_sums = X.sum(axis=1, keepdims=True)  # shape (1, n_cells)
X /= row_sums  # broadcast divide
X *= 10000

# 3) log2(x + 1) transform (in-place)
np.log2(X + 1, out=X)


array([[0.       , 4.8778605, 8.59544  , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 6.76173  , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [193]:
len(train_genes)

768

In [194]:
len(test_genes)

802


The gene list obtained from the train set filtering steps is used to mask the test set. In the original ACTINN implementation, genes were filtered using both the training and test sets. To avoid potential information leakage, we instead apply filtering based solely on the training set.

In [198]:
test_gene_mask=[i in train_genes for i in test_genes]
X = X[:,test_gene_mask]

In [199]:
test_dataset = dc.data.NumpyDataset(X=np.array(X,dtype=np.float32))

In [200]:
test_dataset

<NumpyDataset X.shape: (1000, 768), y.shape: (1000, 1), w.shape: (1000, 1), task_names: [0]>

ACTINN implemented with deepchem Model class as wrapper

In [ ]:
import torch.nn as nn
import torch
from deepchem.models.torch_models import TorchModel
from deepchem.models.losses import SparseSoftmaxCrossEntropy
from deepchem.models.optimizers import Adam
from deepchem.models.optimizers import ExponentialDecay


class ActinnClassifier(nn.Module):

    def __init__(self, output_dim=None, input_size=None):

        if output_dim == None or input_size == None:
            raise ValueError('Must explicitly declare input dim (num features) and output dim (number of classes)')

        super(ActinnClassifier, self).__init__()
        self.inp_dim = input_size
        self.out_dim = output_dim

        # feed forward layers
        self.classifier_sequential = nn.Sequential(
                                        nn.Linear(self.inp_dim, 100),
                                        nn.ReLU(),

                                        nn.Linear(100, 50),
                                        nn.ReLU(),

                                        nn.Linear(50, 25),
                                        nn.ReLU(),

                                        nn.Linear(25, output_dim)
                                        )

    def forward(self, x):
        """
        Forward pass of the classifier
        """
        out = self.classifier_sequential(x)
        return out

class ACTINNModel(TorchModel):
    def __init__(self, output_dim = None, input_size = None, **kwargs):

        self.model = ActinnClassifier(output_dim, input_size)
        self.criterion = SparseSoftmaxCrossEntropy()
        print('model', self.model)
        cf_optimizer = Adam(learning_rate=0.0001,
                            beta1=0.9,
                            beta2=0.999,
                            epsilon=1e-08,
                            weight_decay=0.005,
                            )

        cf_decayRate = 0.95
        cf_lr_scheduler = ExponentialDecay(initial_rate=0.0001, decay_rate=cf_decayRate, decay_steps=1000)
        super(ACTINNModel,
              self).__init__(self.model,
                             loss=self.criterion,
                             optimizer=cf_optimizer,
                             learning_rate=cf_lr_scheduler,
                             output_types=['prediction'],
                             **kwargs)


## Model training and evaluation

In [207]:
model = ACTINNModel(output_dim= n_types,input_size= len(train_genes))

model ActinnClassifier(
  (classifier_sequential): Sequential(
    (0): Linear(in_features=768, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=25, bias=True)
    (5): ReLU()
    (6): Linear(in_features=25, out_features=5, bias=True)
  )
)


In [208]:
model.fit(train_dataset, nb_epoch=5)
logits= model.predict(test_dataset)

In [209]:
import torch.nn.functional as F

In [ ]:
logits_tensor = torch.from_numpy(logits)
probabilities = F.softmax(logits_tensor, dim=1)
predictions = np.argmax(probabilities, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

# Compute accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

### References
[1] Tang, F., Barbacioru, C., Wang, Y., Nordman, E., Lee, C., Xu, N., ... & Surani, M. A. (2009). mRNA-Seq whole-transcriptome analysis of a single cell. Nature methods, 6(5), 377-382.

[2] Alberts, B., Johnson, A., Lewis, J., Morgan, D., Raff, M., Roberts, K., & Walter, P. (2014). Molecular Biology of the Cell (6th ed.). Garland Science.

[3] https://onlinelibrary.wiley.com/doi/full/10.1002/qub2.78

[4] [ACTINN: automated identification of cell types in single cell RNA sequencing](https://academic.oup.com/bioinformatics/article/36/2/533/5540320)

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Discord
The DeepChem [Discord](https://discord.gg/cGzwCdrUqS) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!